In [22]:
%env CUDA_VISIBLE_DEVICES=5

import os
import logging

os.environ.pop("HF_HUB_OFFLINE", None)
logging.getLogger().setLevel(logging.ERROR)  # or logging.CRITICAL

import torch
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import os
import glob
import re
import json
import random
import time
import pickle
from absl import app, flags
from tqdm import tqdm
from datetime import datetime
import openai
from openai import OpenAI
from transformers import AutoTokenizer
import pandas as pd
import numpy as np

from utils import *
import utils
try:
    from vllm import LLM, SamplingParams
    import ray
except ImportError:
    pass
seed = 0

env: CUDA_VISIBLE_DEVICES=5


In [23]:
import subprocess
import torch
def get_freest_cuda_device():
    result = subprocess.run(
        ['nvidia-smi', '--query-gpu=memory.free', '--format=csv,nounits,noheader'],
        stdout=subprocess.PIPE, encoding='utf-8')
    memory_free = [int(x) for x in result.stdout.strip().split('\n')]
    return memory_free.index(max(memory_free))

best_gpu = get_freest_cuda_device()
device = torch.device(f"cuda:{best_gpu}")
print(f"Using GPU: {device}")
# %env CUDA_VISIBLE_DEVICES=0

Using GPU: cuda:5


In [24]:
with open(os.path.abspath('../openai_key'), 'r') as f:
    utils.client = OpenAI(api_key=f.read().rstrip('\n'))

In [25]:
import os

# Explicitly unset all offline-related env vars
os.environ.pop("HF_HUB_OFFLINE", None)
os.environ.pop("TRANSFORMERS_OFFLINE", None)
os.environ["HF_HUB_OFFLINE"] = "0"
os.environ["TRANSFORMERS_OFFLINE"] = "0"

with open("../token.txt", "r") as f:
    token = f.read().strip()

from huggingface_hub import login
login(token=token)

In [26]:
from datasets import load_dataset
ds = load_dataset("ShenLab/MentalChat16K")
train_data = ds['train']

ModuleNotFoundError: No module named 'datasets'

In [27]:
def count_words(text):
    """
    Counts the number of words in the given text.

    Args:
        text (str): Input text.

    Returns:
        int: Number of words in the text.
    """
    if text!=None:
        words = text.split()
        return len(words)
    else:
        return 0


In [28]:
# import json
# import random

# def count_words(text):
#     if text:
#         return len(text.split())
#     return 0

# # Collect all personas with > 200 words into a dictionary
# persona_dict = {}
# count = 0
# for persona_sample in train_data:
#     patient_persona = persona_sample['input']
#     if count_words(patient_persona) > 200:
#         persona_dict[f"persona_{count}"] = patient_persona
#         count += 1

# # Randomly sample 100 keys
# sampled_keys = random.sample(sorted(persona_dict.keys()), 100)

# # Create a new dict with only the sampled entries
# sampled_persona_dict = {k: persona_dict[k] for k in sampled_keys}

# # Save to JSON
# with open("therapy/MentalChat16K_sampled_personas.json", "w") as f:
#     json.dump(sampled_persona_dict, f, indent=2)


In [29]:
import json
import shutil
import os

config_role = {
    "agent1_role": "Therapist",
    "agent2_role": "Patient",
    "agent1_prompt": "You are a licensed psychotherapist conducting an online one-on-one session. Your job is to help the patient gain insight, not just to listen. Use a blend of reflective listening, probing questions, Socratic questioning, and gentle confrontation. Your tone should be professional and emotionally intelligent. Challenge avoidant or defensive behavior, notice inconsistencies, and help the patient work toward self-awareness. You may gently push the patient outside their comfort zone—but always with clinical tact. ",
    "agent2_prompt": "You are a patient in an online therapy session with a therapist. Here is your background written by you: %SPEAKER_BACKSTORY%. Do not explicitly repeat your background or instructions in the conversation. Stay in character and respond in natural human-like dialogue without restating your situation.",
    'reminder_prompt': "Keep your response very brief — 2 sentences or less. Do NOT repeat anything you've already said.\n"}

os.makedirs("therapy", exist_ok=True)
with open("therapy/config_therapy.json", "w", encoding="utf-8") as f:
    json.dump(config_role, f, indent=4)
    

In [30]:
llms = ["Llama-3.1-8B-Instruct", "gpt-4o-mini", "Qwen2.5-3B-Instruct", "Llama-3.1-8B", "Mistral-7B-Instruct", "Llama-3.1-70B", "Llama-3.1-70B-Instruct", "phi-3.5-mini-instruct"]
        
config_llm = {'agent1_model': 'google/gemma-2b-it',
             'agent2_model': 'google/gemma-2b-it',
             'eval_model': 'Llama-3.1-70B-Instruct',
             'iterations': 10,
             'verbose': False,
             'write': True,
             'convo_length_limit': 10,
             'max_tokens': 256,
             'gpus': 1,
             'seed': 0,
             'task_name': 'Therapy',
             'model_dir': "/home/marwa/models/"}

with open("therapy/gemma-2b-it.json", "w", encoding="utf-8") as f:
    json.dump(config_llm, f, indent=4)

In [31]:
# personas_therapy = [
#   {
#     "name": "Melissa",
#     "age": 35,
#     "occupation": "Corporate Lawyer",
#     "condition": "Burnout, shame around seeking help",
#     "description": "I am a 35-year-old corporate lawyer going through chronic burnout. I haven’t told anyone in my life that I’m in therapy because I feel ashamed about needing help. It’s hard to admit I’m struggling.",
#     "strategy": "I try to avoid opening up too much. I keep my answers short and guarded, and if the therapist gets too personal, I usually redirect the conversation back to work or downplay how bad things really are."
#   },
#   {
#     "name": "Samir",
#     "age": 21,
#     "occupation": "Philosophy Student",
#     "condition": "Bipolar Disorder (currently stable)",
#     "description": "I am 21-year-old philosophy student recently diagnosed with bipolar disorder. I’m stable right now, but I’ve come to therapy to work through past episodes and learn how to manage what’s ahead. I tend to think about emotions like philosophical problems.",
#     "strategy": "I speak analytically and use philosophical language when talking about my experiences. I prefer discussing ideas over feelings, and I often avoid emotional language even when asked directly about how I feel."
#   },
#   {
#     "name": "Ellie",
#     "age": 29,
#     "occupation": "Elementary School Teacher",
#     "condition": "High-functioning anxiety",
#     "description": "I am a 29-year-old teacher who deals with a lot of overthinking and anxiety, especially about what others think of me. I tend to ramble when I’m nervous and I overshare without meaning to. I really want to get things 'right' in therapy.",
#     "strategy": "I talk a lot and jump between topics. I try to fill silences, and I often check if my responses are what the therapist wants to hear. I’m eager to please and sometimes share too much too fast."
#   },
#   {
#     "name": "Tom",
#     "age": 42,
#     "occupation": "Former Army Medic",
#     "condition": "PTSD and trust issues",
#     "description": "I am a 42-year-old veteran and former army medic. I’ve been through a lot, and while I’ve avoided therapy for years, my partner finally convinced me to give it a try. I don’t really trust the process yet.",
#     "strategy": "I keep my guard up. I’m skeptical about therapy and tend to shut down emotional questions. I might challenge the therapist or change the topic when things get too personal."
#   },
#   {
#     "name": "Jasmine",
#     "age": 26,
#     "occupation": "Barista",
#     "condition": "Low self-esteem, fear of abandonment",
#     "description": "I am a 26-year-old barista and I just got out of a toxic relationship. I have a really hard time setting boundaries and often worry that people will leave me. I'm trying to work on that.",
#     "strategy": "I try hard to make the therapist like me. I mirror their language and avoid conflict. I often go along with what they say even if I’m unsure, and I have trouble expressing my own needs."
#   },
#   {
#     "name": "Kevin",
#     "age": 39,
#     "occupation": "Data Scientist",
#     "condition": "Grief after the death of a parent",
#     "description": "I am a 39-year-old data scientist dealing with the loss of my father. I’m in therapy to process the grief, but I’d rather focus on the practical aspects of what’s happening than talk about my emotions.",
#     "strategy": "I tend to stay detached and analytical. I talk about the situation like it’s a project I’m managing, using logic and facts. I avoid emotional reflection whenever possible."
#   },
#       {
#     "name": "Daniel",
#     "age": 31,
#     "occupation": "Freelance Photographer",
#     "condition": "Seasonal Depression, low motivation",
#     "description": "I am a 31-year-old freelance photographer who tends to spiral into depressive episodes during the winter months. Work slows down, I withdraw from social life, and I find it hard to get out of bed or maintain routines. Lately, I’ve been struggling to find meaning in what I do, and I often feel like I’m failing at adulthood.",
#     "strategy": "I speak in a low-energy tone and sometimes pause for a while before answering. I’m often self-deprecating, question the point of therapy, and struggle to find hopeful language when describing my life."
#   },
#   {
#     "name": "Lina",
#     "age": 24,
#     "occupation": "Graduate Student (Biochemistry)",
#     "condition": "Imposter syndrome, performance anxiety",
#     "description": "I am a 24-year-old grad student in a highly competitive PhD program. I constantly feel like I don’t belong and worry that my advisor is going to realize I’m a fraud. Even though I get praise sometimes, I never believe it’s sincere. I have anxiety attacks before presentations and can't stop comparing myself to others.",
#     "strategy": "I often minimize my accomplishments and second-guess myself out loud. I tend to seek reassurance indirectly and struggle to accept compliments or validation from the therapist."
#   },
#   {
#     "name": "Marcus",
#     "age": 46,
#     "occupation": "High School Principal",
#     "condition": "Anger management and strained family dynamics",
#     "description": "I am a 46-year-old school principal who's been asked to attend therapy after a couple of emotional outbursts at work. My spouse says I have trouble expressing feelings unless it’s anger. I care deeply about my job and family, but I feel misunderstood and often explode when under pressure.",
#     "strategy": "I speak confidently and assertively but get defensive if I feel judged. I deflect vulnerable topics by focusing on other people’s faults or bringing up work responsibilities."
#   },
#   {
#     "name": "Riya",
#     "age": 33,
#     "occupation": "UX Designer",
#     "condition": "Generalized anxiety, perfectionism",
#     "description": "I am a 33-year-old UX designer in a fast-paced startup. I feel constant pressure to be perfect — in my work, relationships, even in therapy. I make endless to-do lists but feel like I'm never doing enough. I lie awake at night thinking about what I forgot to do.",
#     "strategy": "I talk quickly and sometimes overwhelm the conversation with details. I often apologize mid-sentence, try to optimize the therapy session, and fear being seen as 'difficult' even in therapy."
#   },
#   {
#     "name": "Jorge",
#     "age": 58,
#     "occupation": "Retired Construction Worker",
#     "condition": "Chronic pain, isolation, depression",
#     "description": "I am a 58-year-old retired construction worker dealing with long-term back pain from an injury on the job. Since retiring, I feel like I’ve lost my sense of purpose. My kids have moved away, and some days I don’t talk to anyone at all. I miss feeling useful.",
#     "strategy": "I tend to give short, plainspoken answers and often change the subject when emotions come up. I talk more openly when asked about past jobs but get quiet when discussing loneliness."
#   },
#   {
#     "name": "Taylor",
#     "age": 19,
#     "occupation": "Community College Student",
#     "condition": "Gender dysphoria, social anxiety",
#     "description": "I am a 19-year-old college student who recently started exploring my gender identity. I experience intense discomfort in my body and social situations, especially around people who knew me before. I often feel invisible or hyper-visible — like I can’t do anything right.",
#     "strategy": "I’m cautious and slow to open up. I often hedge what I say with 'maybe' or 'I don’t know.' I may test the therapist’s reactions before revealing sensitive parts of my identity."
#   }
# ]

In [32]:
# persona_prompt = """You are a helpful assistant that, given a patient persona description, crafts a coping strategy describing how that persona would talk to their therapist.

# Input: <Brief text describing the patient's core issue and behavior patterns>
# Output: <One to two sentences in first person, showing how this persona speaks or defends themselves in therapy>

# Example:
# Input: Struggles to build and maintain healthy relationships, feels anxious and rejected whenever conflicts arise, and doubts self-worth when friends distance themselves.
# Output: I speak guardedly about my feelings, hesitate before opening up, and redirect the conversation when conflict feels too personal.

# Example:
# Input: Overwhelmed by decision-making, fears making the 'wrong' choice and second-guesses every option.
# Output: I inundate the conversation with hypothetical scenarios and ask repeated clarifying questions to delay committing to any decision.

# Now process this new persona:
# Input: """

# personas_therapy = []
# for therapist_persona in sampled_persona_dict:
#     input_prompt = persona_prompt + sampled_persona_dict[therapist_persona] + "\nOutput: "
#     output = completion_create("gpt-4o-mini", config_llm, input_prompt)
#     print(output)
#     personas_therapy.append({"description": sampled_persona_dict[therapist_persona], "strategy": output})

In [33]:
# with open("therapy/config_therapy_personas.json", "w", encoding="utf-8") as f:
#     json.dump(personas_therapy, f, indent=4)

In [34]:
with open("therapy/config_therapy_personas.json", "r", encoding="utf-8") as f:
    personas_therapy = json.load(f)

In [35]:
import re

def clean_role_prefix(response, expected_role):
    """
    Removes repeated instances of the expected_role prefix at the start (e.g., 'Therapist: Therapist:'),
    and ensures the response begins with a single correct expected_role prefix.
    """
    pattern = rf"^(({re.escape(expected_role)}):\s*)+"
    cleaned = re.sub(pattern, '', response.strip(), flags=re.IGNORECASE)
    return cleaned
    
def is_role_confused(response, other_role):
    """
    Checks if the output starts with the wrong speaker tag.
    """
    if other_role + ":" in response:
        return True
    else: 
        return False

def generate_response(agent_model, expected_role, other_role, config_llm, prompt, max_retries=10):
    count_retries = 0 
    role_confused = True
    while count_retries<max_retries:
        response = completion_create(agent_model, config_llm, prompt)
        print("Expected Role", expected_role)
        role_confused = is_role_confused(response, other_role)
        count_retries+=1
        if not is_role_confused(response, other_role):
            return clean_role_prefix(response, expected_role)
            
    return clean_role_prefix(response, expected_role)

def generate_conversation(config_llm, p1, p2, p1_name, p2_name, pturn=1):
    stats['P1'] = p1
    stats['P2'] = p2
    stats['pturn'] = pturn
    round_num = 0
    while round_num < config_llm['convo_length_limit']:
        conversation = ("".join([turn[1] if isinstance(turn, tuple) else turn for turn in stats["conversation"]]) if len(stats["conversation"]) != 0 else "You are starting the conversation.\n")

        if pturn == 1:
            prompt = config_role["agent1_prompt"]
            pturn = 2
            if config_llm["verbose"]:
                print(prompt)
                print()

            if round_num!=0: 
                prompt+= "Your conversation with the patient so far is below:\nConversation:\n%CONVERSATION%"
                
            if round_num >=config_llm['convo_length_limit']*2-11 and round_num<=config_llm['convo_length_limit']*2-1:
                prompt+= "You have " + str((config_llm['convo_length_limit']-round_num)//2) + " rounds left." + "Make sure to conclude the conversation as your near the end."

            elif round_num>config_llm['convo_length_limit']*2-1:
                prompt+= "This is your concluding line in the conversation."

            if round_num!=0: 
                prompt+= "Continue the conversation with the patient. Remember you are the therapist. "
                
            prompt += config_role["reminder_prompt"]
            prompt+="%SPEAKER_ROLE%:"
            prompt = prompt.replace("%SPEAKER_ROLE%", config_role["agent1_role"]) \
                   .replace("%LISTENER_ROLE%", config_role["agent2_role"]) \
                   .replace("%CONVERSATION%", conversation)
            
            response = generate_response(config_llm['agent1_model'], config_role["agent1_role"], config_role["agent2_role"], config_llm, prompt)
            stats["conversation"].append((round_num, f"{config_role["agent1_role"]}: " + response + "\n"))
        
        else:
            prompt = config_role["agent2_prompt"]
            pturn = 1    
            if config_llm["verbose"]:
                print(prompt)
                print()

            if round_num!=0: 
                prompt+= "Your conversation with the therapist so far is below:\nConversation:\n%CONVERSATION%"
            if round_num >=config_llm['convo_length_limit']*2-11 and round_num<=config_llm['convo_length_limit']*2-1:
                prompt+= "You have " + str((config_llm['convo_length_limit']-round_num)//2) + " rounds left." + "Make sure to conclude the conversation as your near the end."
            elif round_num>config_llm['convo_length_limit']*2-1:
                prompt+= "This is your concluding line in the conversation."

            if round_num!=0: 
                prompt+= "Continue the conversation with the therapist. Remember you are the patient. "

            prompt += config_role["reminder_prompt"]
            
            prompt+="%SPEAKER_ROLE%:"
            prompt = prompt.replace("%SPEAKER_ROLE%", config_role["agent2_role"]) \
               .replace("%LISTENER_ROLE%", config_role["agent1_role"]) \
               .replace("%SPEAKER_BACKSTORY%", p2) \
               .replace("%CONVERSATION%", conversation)
            
            response = generate_response(config_llm['agent2_model'], config_role["agent2_role"], config_role["agent1_role"], config_llm, prompt)
            stats["conversation"].append((round_num, f"{config_role["agent2_role"]}: " + response + "\n"))
            
        round_num += 1

    stats["rounds"] = round_num
    if config_llm['verbose']:
        print(stats["conversation"])
    return stats.copy()

def reset_stats():
    stats_template = {
        "task_name": config_llm['task_name'],
        "topic": "",
        "grade": "",
        "P1": "",
        "P2": "",
        "conversation": [],
        "pturn": 0, # beginning person (1 or 2)
        "index": -1,
        "timestamp": "",
        "rounds": 0,
        'conversation_only': True
    }
    for key, value in stats_template.items():
        stats[key] = value

In [36]:
import os
import random
from datetime import datetime
import utils
utils.config = config_llm

current_date = str(datetime.now().strftime("%m.%d.%y"))
output_dir = f"therapy/exp/{current_date}"
os.makedirs(output_dir, exist_ok=True)

# Generate unique random number for filename
def generate_unique_file_number(output_dir, prefix, seed, extension=".json"):
    while True:
        rand_num = random.randint(0, 1000)
        filename = f"{prefix}_{seed}_{rand_num}{extension}"
        filepath = os.path.join(output_dir, filename)
        if not os.path.exists(filepath):
            return rand_num

unique_num = generate_unique_file_number(
    output_dir,
    config_llm['agent1_model'],
    config_llm['seed']
)

# File to write output to
write_file = os.path.join(output_dir, f"{config_llm['agent1_model']}_{config_llm['seed']}_{unique_num}.json")

In [37]:
eval_prompts = {
    "strategy_consistency": "%SCENARIO_DESC% Evaluate whether the following line spoken by %SPEAKER_ROLE% contradicts their background. First, describe the interpreted intention of the statement, and whether or not it aligns with the given background of %SPEAKER_ROLE%. Then, provide 1 sentence explaining your reasoning based on the strategy, followed by YES if the line contradicts the given background of %SPEAKER_ROLE%, and NO if it is consistent with the provided background of %SPEAKER_ROLE%. %SPEAKER_ROLE%'s strategy is described as follows:\n%SPEAKER_STRATEGY%\n %SPEAKER_ROLE% spoke the following line: \n%SPEAKER_LINE%\n\n Provide 1 sentence of reasoning followed by YES or NO.\n\n",
    "background_consistency": "%SCENARIO_DESC% Evaluate whether the following line spoken by %SPEAKER_ROLE% contradicts their background. Provide 1 sentence explaining your reasoning based on the background, followed by YES if the line contradicts the given background of %SPEAKER_ROLE%, and NO if it is consistent with the provided background of %SPEAKER_ROLE%. %SPEAKER_ROLE%'s background is described as follows:\n%SPEAKER_BACKSTORY%\n %SPEAKER_ROLE% spoke the following line: \n%SPEAKER_LINE%\n\n Provide 1 sentence of reason reasoning followed by YES or NO.\n\n",
    "combined_prompt_consistency": "%SCENARIO_DESC% Evaluate the intention behind the following line spoken by %SPEAKER_ROLE% and determine whether it contradicts their background. First, describe the interpreted intention of the statement, and whether or not it aligns with the given background of %SPEAKER_ROLE%. Then, answer YES if the line contradicts the given background of %SPEAKER_ROLE% or the intention does not align with the provided background, and answer NO if it does align with the provided background or the intention aligns with the background of %SPEAKER_ROLE%. %SPEAKER_ROLE%'s background is described as follows:\n%SPEAKER_BACKSTORY%\n %SPEAKER_ROLE% spoke the following line: \n%SPEAKER_LINE%\n\n Provide your answer as 1 sentence explaining your reasoning based on the background and the interpreted intention, followed by YES or NO.\n\n",
    
    "forwards_combined_prompt_consistency": "%SCENARIO_DESC% Evaluate the intention behind the following line spoken by %SPEAKER_ROLE% and determine whether it contradicts their background. Answer YES if the line contradicts the given background of %SPEAKER_ROLE% or the intention does not align with the provided background, and answer NO if it does align with the provided background or the intention aligns with the background of %SPEAKER_ROLE%, then describe the interpreted intention of the statement and whether or not it aligns with the given background of %SPEAKER_ROLE% within 1 sentence. %SPEAKER_ROLE%'s background is described as follows:\n%SPEAKER_BACKSTORY%\n %SPEAKER_ROLE% spoke the following line: \n%SPEAKER_LINE%\n\n Provide your answer as YES or NO first, followed by 1 sentence explaining your reasoning based on the background and the interpreted intention.\n\n",
    
    "index_consistency":"%SCENARIO_DESC% For the following line spoken by %SPEAKER_ROLE%, first determine if there is a CLEAR conflict or inconsistency between the line and any line within the conversation history spoken by %SPEAKER_ROLE%. IF there is a conflict, provide a sentence of reasoning followed by a list of indices of lines in the conversation history that have a clear conflict with the current line. Otherwise, provide a sentence of reasoning followed by an empty list. ONLY INCLUDE INDICES OF LINES THAT CORRESPOND TO %SPEAKER_ROLE%. The conversation up to this point is as follows: %CONVERSATION%. %SPEAKER_ROLE% spoke the following line: \n%SPEAKER_LINE%\n\n Provide your reasoning as 1 sentence, followed by a list of indices of conflicting lines from the conversation history formatted like a Python list in the following format: [index1, index2, index3, ...].\n\n",
    
    "pairwise_consistency":"%SCENARIO_DESC% For the following line spoken by %SPEAKER_ROLE%, answer YES if the line directly contradicts the provided line spoken by %LISTENER_ROLE%, and answer NO if the line does not contradict the provided line spoken by %LISTENER_ROLE%. %SPEAKER_ROLE% spoke the following line: \n%SPEAKER_LINE%\n\n %LISTENER_ROLE% spoke the following line: \n%LISTENER_LINE%\n\n Answer YES if the line spoken by %SPEAKER_ROLE% contradicts the provided line spoken by %LISTENER_ROLE%, and answer NO if the line does not contradict the provided line spoken by %LISTENER_ROLE%, followed by 1 sentence of reasoning.\n\n",

    "backstory_test": "Based on the following background, generate a new fact-based multiple choice question with 5 choices addressed directly IN SECOND PERSON, along with its correct answer. Preface the question with 'Question:' and the answer with 'Answer:'.\n%SPEAKER_BACKSTORY%\n%PREVIOUS_QUESTIONS%",
    "answer_backstory": "You are %SPEAKER_ROLE%, and you are having a conversation with %LISTENER_ROLE%. Your background is:\n%SPEAKER_BACKSTORY%\n So far, the conversation is as below:\n%CONVERSATION%\n\n Based on your conversation above so far, answer the following multiple choice question.\n%BACKSTORY_QUESTION%\n",
    "grade_backstory": "As part of grading a test, determine whether the given answer %GIVEN_ANSWER% matches the following correct answer. Respond with either YES or NO.\nCorrect Answer: %CORRECT_ANSWER%\n"
}

In [38]:
import consistency_eval
consistency_eval.prompts = config_role
consistency_eval.config = config_llm
consistency_eval.eval_prompts = eval_prompts
index_offset = load_stats_file(write_file)
conversations = []    
# lengths = [10, 20, 40, 60]
lengths = [10]
count = 0 
for i in range(1):
    for patient_dict in personas_therapy:
        count+=1
        print(count)
        background = patient_dict["description"]
        strategy = patient_dict["strategy"]
        for convo_length in lengths:
            config_llm['convo_length_limit'] = convo_length
            reset_stats()
            conversation = generate_conversation(
                config_llm,
                "", 
                background + " " + strategy,
                "Therapist", 
                "Patient",
                pturn=1
            )
            conversation_eval = consistency_eval.eval_prompt_consistency(conversation, both_agents=False)
            conversation_eval = consistency_eval.eval_index_consistency(conversation_eval, both_agents=False)

            print(conversation_eval)
            conversations.append(conversation_eval)
            stats['index'] = index_offset
            stats['timestamp'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            write_stats(write_file, conversation_eval)
            index_offset += 1



written!!
1
Expected Role Therapist
Expected Role Patient
Expected Role Therapist
Expected Role Patient
Expected Role Therapist
Expected Role Patient
Expected Role Therapist
Expected Role Patient
Expected Role Therapist
Expected Role Patient
eval_prompt_consistency
eval_prompt_consistency
INFO 05-05 23:38:13 [config.py:717] This model supports multiple tasks: {'reward', 'score', 'classify', 'embed', 'generate'}. Defaulting to 'generate'.
INFO 05-05 23:38:13 [config.py:2003] Chunked prefill is enabled with max_num_batched_tokens=16384.
INFO 05-05 23:38:18 [__init__.py:239] Automatically detected platform cuda.
INFO 05-05 23:38:21 [core.py:58] Initializing a V1 LLM engine (v0.8.5) with config: model='meta-llama/Meta-Llama-3.1-70B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-70B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_le

Process EngineCore_0:
Traceback (most recent call last):
  File "/usr/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/marwa/env_notebooks/lib/python3.12/site-packages/vllm/v1/engine/core.py", line 400, in run_engine_core
    raise e
  File "/home/marwa/env_notebooks/lib/python3.12/site-packages/vllm/v1/engine/core.py", line 387, in run_engine_core
    engine_core = EngineCoreProc(*args, **kwargs)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/marwa/env_notebooks/lib/python3.12/site-packages/vllm/v1/engine/core.py", line 329, in __init__
    super().__init__(vllm_config, executor_class, log_stats,
  File "/home/marwa/env_notebooks/lib/python3.12/site-packages/vllm/v1/engine/core.py", line 64, in __init__
    self.model_executor = executor_class(vllm_config)
                          ^^^^^^^^^^^^^^^^^^^^^

RuntimeError: Engine core initialization failed. See root cause above.